# Optimizing with CMA-ES in Jax

This notebook shows how to use QDax to find performing parameters on the Rastrigin problem with [CMA-ES](https://arxiv.org/pdf/1604.00772.pdf). It can be run locally or on Google Colab. We recommand to use a GPU. This notebook will show:

- how to define the problem
- how to create a cma-es optimizer
- how to launch a certain number of optimizing steps
- how to visualise the optimization process

In [ ]:
import jax
import jax.numpy as jnp

import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

from qdax.core.cmaes import CMAES

## Set the hyperparameters

In [ ]:
#@title Hyperparameters
#@markdown ---
num_iterations = 500 #@param {type:"integer"}
num_dimensions = 20 #100 #1000 #@param {type:"integer"} # try 20 and 100
batch_size = 500 #@param {type:"integer"}
num_best = 250 #@param {type:"integer"}
sigma_g = 5. # 0.5 #@param {type:"number"}
init_step_size = 1. #@param {type:"number"}
minval = -5.12 #@param {type:"number"}
#@markdown ---

## Define the fitness function - choose rastrigin or sphere

In [ ]:
def rastrigin_scoring(x: jnp.ndarray):
    first_term = 10 * x.shape[-1]
    second_term = jnp.sum((x + minval * 0.4) ** 2 - 10 * jnp.cos(2 * jnp.pi * (x + minval * 0.4)))
    return -(first_term + second_term)

def sphere_scoring(x: jnp.ndarray):
    return -jnp.sum((x + minval * 0.4) * (x + minval * 0.4), axis=-1)

# fitness_fn = sphere_scoring
fitness_fn = jax.vmap(rastrigin_scoring)

## Define a CMA-ES optimizer instance

In [ ]:
cmaes = CMAES(
    population_size=batch_size,
    num_best=num_best,
    search_dim=num_dimensions,
    fitness_function=fitness_fn,
    mean_init=jnp.zeros((num_dimensions,)),
    init_sigma=sigma_g,
    init_step_size=init_step_size,
    delay_eigen_decomposition=True,
)

## Init the CMA-ES optimizer state

In [ ]:
state = cmaes.init()
key = jax.random.PRNGKey(0)

## Run optimization iterations

In [ ]:
%%time

means = [state.mean]
covs = [state.cov_matrix]

for _ in range(num_iterations):
    # sample
    samples, key = cmaes.sample(state, key)
    
    # udpate
    state = cmaes.update(state, samples)
    
    # check stop condition
    stop_condition = cmaes.stop_condition(state)

    if stop_condition:
        break
    
    # store data for plotting
    means.append(state.mean)
    covs.append(state.cov_matrix)

## Check final fitnesses and distribution mean

In [ ]:
# checking final fitness values
fitnesses = fitness_fn(samples)

print("Min fitness in the final population: ", jnp.min(fitnesses))
print("Mean fitness in the final population: ", jnp.mean(fitnesses))
print("Max fitness in the final population: ", jnp.max(fitnesses))

# checking mean of the final distribution
print("Final mean of the distribution: \n", means[-1])
# print("Final covariance matrix of the distribution: ", covs[-1])

## Visualization of the optimization trajectory

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

# sample points to show fitness landscape
x = jax.random.uniform(key, minval=-4, maxval=8, shape=(100000, 2))
f_x = fitness_fn(x)

# plot fitness landscape
points = ax.scatter(x[:, 0], x[:, 1], c=f_x, s=0.1)
fig.colorbar(points)

# plot cma-es trajectory
traj_min = 0
traj_max = 500
for mean, cov in zip(means[traj_min:traj_max], covs[traj_min:traj_max]):
    ellipse = Ellipse((mean[0], mean[1]), cov[0, 0], cov[1, 1], fill=False, color='k', ls='--')
    ax.add_patch(ellipse)
    ax.plot(mean[0], mean[1], color='k', marker='x')
    
ax.set_title(f"Optimization trajectory of CMA-ES between step {traj_min} and step {traj_max}")
plt.show()